<a href="https://colab.research.google.com/github/woo13sd/rupture-theory/blob/main/notebooks/Rupture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# ✅ Environment Setup Cell 1: Clone CAMB and install dependencies
!apt-get install -y gfortran
!pip install --quiet numpy scipy matplotlib

# Clone CAMB from GitHub (or your own fork if customized)
!git clone https://github.com/cmbant/CAMB.git /content/CAMB

# Move into CAMB directory and build
%cd /content/CAMB
!make clean
!make all

# Verify build succeeded
!ls -lh camb
print("✅ CAMB cloned and compiled.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gfortran is already the newest version (4:11.2.0-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Cloning into '/content/CAMB'...
remote: Enumerating objects: 8764, done.
remote: Counting objects: 100% (2859/2859), done.
remote: Compressing objects: 100% (425/425), done.
remote: Total 8764 (delta 2687), reused 2434 (delta 2434), pack-reused 5905 (from 3)
Receiving objects: 100% (8764/8764), 123.09 MiB | 24.09 MiB/s, done.
Resolving deltas: 100% (6426/6426), done.
/content/CAMB
make: *** No rule to make target 'clean'.  Stop.
make: *** No rule to make target 'all'.  Stop.
total 708K
-rw-r--r-- 1 root root  31K Jul 28 21:00 baseconfig.py
-rw-r--r-- 1 root root 9.9K Jul 28 21:00 bbn.py
-rw-r--r-- 1 root root  803 Jul 28 21:00 bispectrum.py
-rw-r--r-- 1 root root  17K Jul 28 21:00 camb.py
-rw-r--r-- 1 root root 1.5K Jul 28 21:00 _command_line.py
-rw-r--r-- 1 root root 

In [68]:
# ✅ Environment Setup Cell 2: Replace Fortran modules and rebuild CAMB

# Define paths (adjust if your files are stored elsewhere)
custom_dir = "/content/rupture-theory/fortran"  # Update if your repo uses a different path
camb_dir = "/content/CAMB"

# Overwrite the CAMB Fortran files with your custom physics
!cp {custom_dir}/initialpower.f90 {camb_dir}/initialpower.f90
!cp {custom_dir}/reionization.f90 {camb_dir}/reionization.f90

# Rebuild CAMB with the custom Fortran files
%cd {camb_dir}
!make clean
!make all

# Verify build and acknowledge
!ls -lh camb
print("✅ Custom Fortran physics modules injected and CAMB recompiled.")

cp: cannot stat '/content/rupture-theory/fortran/initialpower.f90': No such file or directory
cp: cannot stat '/content/rupture-theory/fortran/reionization.f90': No such file or directory
/content/CAMB
make: *** No rule to make target 'clean'.  Stop.
make: *** No rule to make target 'all'.  Stop.
total 708K
-rw-r--r-- 1 root root  31K Jul 28 21:00 baseconfig.py
-rw-r--r-- 1 root root 9.9K Jul 28 21:00 bbn.py
-rw-r--r-- 1 root root  803 Jul 28 21:00 bispectrum.py
-rw-r--r-- 1 root root  17K Jul 28 21:00 camb.py
-rw-r--r-- 1 root root 1.5K Jul 28 21:00 _command_line.py
-rw-r--r-- 1 root root 4.7K Jul 28 21:00 _compilers.py
-rw-r--r-- 1 root root 2.5K Jul 28 21:00 _config.py
-rw-r--r-- 1 root root 1.7K Jul 28 21:00 constants.py
-rw-r--r-- 1 root root  32K Jul 28 21:00 correlations.py
-rw-r--r-- 1 root root 8.6K Jul 28 21:00 dark_energy.py
-rw-r--r-- 1 root root 9.9K Jul 28 21:00 emission_angle.py
-rw-r--r-- 1 root root 6.9K Jul 28 21:00 initialpower.py
-rw-r--r-- 1 root root 1.3K Jul 28 2

In [69]:
%cd /content/CAMB
!make clean
!make

/content/CAMB
make: *** No rule to make target 'clean'.  Stop.
make: *** No targets specified and no makefile found.  Stop.
